In [ ]:
#!pip install dash

task 1

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

spacex_df = pd.read_csv('/content/spacex_launch_dash.csv')

launch_sites = spacex_df['Launch Site'].unique()

site_dropdown = dcc.Dropdown(
    id='site-dropdown',
    options=[
        {'label': 'Tất cả các bệ', 'value': 'ALL'}
    ] + [{'label': site, 'value': site} for site in launch_sites],
    value='ALL',
    placeholder='Chọn bệ phóng',
    searchable=True
)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Dashboard SpaceX'),
    site_dropdown
])

task 2

In [ ]:
# Hàm callback để render biểu đồ tròn thành công
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, names='class', title='Tỷ lệ thành công tổng thể')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', title=f'Tỷ lệ thành công tại {entered_site}')
    return fig

task 3

In [ ]:
payload_slider = dcc.RangeSlider(
    id='payload-slider',
    min=0,
    max=10000,
    step=1000,
    marks={
        0: '0 Kg',
        5000: '5000 Kg',
        10000: '10000 Kg'
    },
    value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]
)

app.layout = html.Div([
    html.H1('Dashboard SpaceX'),
    site_dropdown,
    dcc.Graph(id='success-pie-chart'),
    html.Label('Chọn phạm vi trọng tải (Kg):'),
    payload_slider
])

task 4

In [ ]:
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def render_scatter_plot(selected_site, payload_range):
    import pandas as pd
    import plotly.express as px

    if selected_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == selected_site) &
                                (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]

    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    fig.update_layout(title='Mối quan hệ giữa trọng tải và kết quả phóng')
    return fig

app.layout = html.Div([
    html.H1('Dashboard SpaceX'),
    site_dropdown,
    dcc.Graph(id='success-pie-chart'),
    html.Label('Chọn phạm vi trọng tải (Kg):'),
    payload_slider,
    dcc.Graph(id='success-payload-scatter-chart')
])

In [ ]:
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>